# NLP EDA

Text Data Exploration Techniques

- [Term Frequency](#Term-Frequency)
- [Ngrams](#Ngrams)
- [Document Length](#Document-Length)
- [Word Cloud](#Word-Cloud)
- [Sentiment Analysis](#Sentiment-Analysis)

## Setup

In [ ]:
from typing import List
import unicodedata
import re

import nltk
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# matplotlib default plotting styles
plt.rc("patch", edgecolor="black", force_edgecolor=True)
plt.rc("axes", grid=True)
plt.rc("grid", linestyle=":", linewidth=0.8, alpha=0.7)
plt.rc("axes.spines", right=False, top=False)
plt.rc("figure", figsize=(11, 8))
plt.rc("font", size=12.0)
plt.rc("hist", bins=25)

def clean(text: str) -> List[str]:
    "a simple function to prepare text data"
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words("english") + ["r", "u", "2", "ltgt"]
    text = (
        unicodedata.normalize("NFKD", text)
        .encode("ascii", "ignore")
        .decode("utf-8", "ignore")
        .lower()
    )
    words = re.sub(r"[^\w\s]", "", text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

df = pd.read_csv("spam_clean.csv")

A quick data summary

In [ ]:
df.shape

In [ ]:
df.isna().sum()

What percentage of the data is spam?

In [ ]:
df.label.value_counts().plot.pie(
    colors=["pink", "lightblue"], explode=(0.15, 0), autopct="%.0f%%"
)
plt.title("Ham vs Spam Distribution")
plt.ylabel("")
plt.xlabel("n = %d" % df.shape[0])

pd.concat(
    [df.label.value_counts(), df.label.value_counts(normalize=True)], axis=1
).set_axis(["n", "percent"], axis=1, inplace=False)

## Term Frequency

In [ ]:
pd.Series(" ".join(df.text).split()).value_counts()

1. one big string for everything, spam, ham
1. lists of strings
1. list of strings -> pandas series so we can value count
1. combine series into single dataframe

In [ ]:
all_text = " ".join(df.text)
spam_text = " ".join(df[df.label == "spam"].text)
ham_text = " ".join(df[df.label == "ham"].text)

In [ ]:
all_words = clean(all_text)
spam_words = clean(spam_text)
ham_words = clean(ham_text)

In [ ]:
all_freq = pd.Series(all_words).value_counts()
spam_freq = pd.Series(spam_words).value_counts()
ham_freq = pd.Series(ham_words).value_counts()

In [ ]:
tf = (
    pd.concat([all_freq, ham_freq, spam_freq], axis=1, sort=True)
    .rename(columns={0: "all", 1: "ham", 2: "spam"})
    .fillna(0)
    .apply(lambda col: col.astype(int))
)

- most common words overall?
- most common spam, ham words?
- any words that uniquely spam or ham?

In [ ]:
tf.sort_values(by="all").tail(10)

## Ngrams

- bigrams + viz most frequent for all, spam, ham
- trigrams, etc

In [ ]:
list(nltk.bigrams('I love the smell of regex in the morning'.split()))

In [ ]:
pd.Series(nltk.bigrams(all_words))

## Document Length

In [ ]:
df["message_length"] = df.text.apply(len)

In [ ]:
sns.boxplot(data=df, y="message_length", x="label")

In [ ]:
df.hist("message_length", by="label", sharex=True, layout=(2, 1), bins=25)

### Number of Words

In [ ]:
df["n_words"] = df.text.str.count(r"\w+")

In [ ]:
df.groupby("label").n_words.describe()

In [ ]:
fig = plt.figure()
ax1 = fig.add_axes([0.1, 0.1, 0.4, 0.8])  # left, bottom, width, height
ax2 = fig.add_axes([0.55, 0.1, 0.4, 0.35])
ax3 = fig.add_axes([0.55, 0.55, 0.4, 0.35], sharex=ax2)
sns.boxplot(data=df, y="n_words", x="label", ax=ax1)
df.hist("n_words", by="label", bins=25, ax=[ax2, ax3])
fig.suptitle("Distribution of Number of Words for Spam and Ham Messages")

## Word Cloud

`WordCloud()` produces an image object, which can be displayed with `plt.imshow`

In [ ]:
from wordcloud import WordCloud

sentence = (
    "Mary had a little lamb, little lamb, little lamb. Its fleece was white as snow."
)
img = WordCloud(background_color="white").generate(sentence)
plt.imshow(img)
plt.axis("off")

do the same with all words, spam and ham

In [ ]:
img = WordCloud(background_color="white").generate(all_text)
plt.imshow(img)
plt.axis("off")

In [ ]:
all_cloud = WordCloud(background_color="white", height=1000, width=400).generate(
    " ".join(all_words)
)
ham_cloud = WordCloud(background_color="white", height=600, width=800).generate(
    " ".join(ham_words)
)
spam_cloud = WordCloud(background_color="white", height=600, width=800).generate(
    " ".join(spam_words)
)

plt.figure(figsize=(10, 8))
axs = [
    plt.axes([0, 0, 0.5, 1]),
    plt.axes([0.5, 0.5, 0.5, 0.5]),
    plt.axes([0.5, 0, 0.5, 0.5]),
]

axs[0].imshow(all_cloud)
axs[1].imshow(ham_cloud)
axs[2].imshow(spam_cloud)

axs[0].set_title("All Words")
axs[1].set_title("Ham")
axs[2].set_title("Spam")

for ax in axs:
    ax.axis("off")

### Word Cloud with Bigrams

- `generate_from_frequencies` + python gymnastics

In [ ]:
frequencies = {
    "Codeup": 10,
    "Bayes": 5,
    "Data Science": 6,
}

img = WordCloud(background_color="white").generate_from_frequencies(frequencies)
plt.figure(figsize=(9, 6))
plt.imshow(img)
plt.axis("off")

In [ ]:
top_20_ham_bigrams = pd.Series(nltk.bigrams(ham_words)).value_counts().head(20)

data = {p1 + " " + p2: v for (p1, p2), v in top_20_ham_bigrams.to_dict().items()}

img = WordCloud(
    background_color="white", width=800, height=400
).generate_from_frequencies(data)

plt.figure(figsize=(8, 4))
plt.imshow(img)
plt.axis("off")

## Sentiment Analysis

A way for us to put a number to indicate whether a document has a positive or
negative sentiment.

### Vader

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

In [ ]:
sia.polarity_scores("Sentiment analysis is very awesome!")

In [ ]:
sia.polarity_scores("I am pretty worried about bad weather this weekend.")

In [ ]:
df["vader_sentiment"] = df.text.apply(lambda txt: sia.polarity_scores(txt)["compound"])

In [ ]:
sns.boxplot(data=df, y="vader_sentiment", x="label")
df.groupby("label").vader_sentiment.describe()

### Afinn

In [ ]:
from afinn import Afinn

sa = Afinn()

In [ ]:
sa.score("Sentiment analysis is very awesome!")

In [ ]:
sa.score("I am pretty worried about bad weather this weekend.")

In [ ]:
df["afinn_sentiment"] = df.text.apply(sa.score)

In [ ]:
sns.boxplot(data=df, y="afinn_sentiment", x="label")
df.groupby("label").afinn_sentiment.describe()

## Further Reading

- [VADER Sentiment Analysis](https://github.com/cjhutto/vaderSentiment)
- [AFINN Sentiment Analysis](https://github.com/fnielsen/afinn)

## Other NLP Libraries

- [spaCy](https://spacy.io/)
- [textacy](https://chartbeat-labs.github.io/textacy/) builds on top of spaCy
- [TextBlob](https://textblob.readthedocs.io/en/dev/)

## Bonus Exercises

After you've worked through the exercises in the curriculum,

- Use sentiment analysis to explore your datasets. Which news category has the highest sentiment? Which has the lowest? Does this match with what you might predict?

- Create a feature named `has_long_words`. This should be either true or false depending on whether or not the message contains a word greater than 5 characters. Use this feature to explore spam v ham. What changes if you change the cutoff from 5 to 8 characters?

- Explore the enron spam email dataset

    Download [the json file located here](https://ds.codeup.com/enron_spam.json.gz) and read it with pandas:
    
    We have done a little preprocessing and acquisition of the [data found in this kaggle competition](https://www.kaggle.com/wanderfj/enron-spam) to make it easier to work with.
    
    ```python
    df = pd.read_json("enron_spam.json.gz")
    ```
    
    Start by focusing just on the `label`, `subject`, and `text` columns.